# Тренировка классификационной модели YOLOv5
Способ fine-tuning модели YOLOv5 и обучение за пределами Roboflow.

**Здесь используется:**
1. Модель классификации [YOLOv5](https://github.com/ultralytics/yolov5)
2. Размеченный, аугументированный и разбитый на train/test/valid датасет Roboflow
3. API-интерфейс Roboflow для получения датасета.
4. Подключение Google Drive для сохранения результатов обучения, валидации и предсказаний модели.


**Порядок реализации:**
1. Установка и подключение необходимых библиотек и модулей
2. Тренировка на произвольных данных
   1. Установка модуля Roboflow и загрузка датасета
   2. Тренировка модели YOLOv5
   3. Копирование результатов в Google Drive
3. Валидация модели (на валидационных данных)
4. Предсказание по произвольным изображениям

## 1. Установка необходимых библиотек и модулей

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/112.6 GB disk)


## 2. Тренировка на произвольных данных

Для тренировки модели, необходимо подготовить размеченный датасет.

Для подготовки датасета мы использовали [Roboflow](https://roboflow.com).

Ссылка на датасет: [https://universe.roboflow.com/mipt4/archistyles/dataset/1](https://universe.roboflow.com/mipt4/archistyles/dataset/1)


> Здесь можно найти подробную инструкцию по [тренировке YOLOv5 классификации на произвольных данных](https://blog.roboflow.com/train-YOLOv5-classification-custom-data).



### 2.1. Загружаем датасет

Структура данных в датасете из изображений должна быть такой:

```
dataset
├── train
│   ├── class-one
│   │   ├── IMG_123.jpg
│   └── class-two
│       ├── IMG_456.jpg
├── valid
│   ├── class-one
│   │   ├── IMG_789.jpg
│   └── class-two
│       ├── IMG_101.jpg
├── test
│   ├── class-one
│   │   ├── IMG_121.jpg
│   └── class-two
│       ├── IMG_341.jpg
```

Структура датасета, созданного нами в Roboflow выглядит так:
![](https://j.leadzilla.ru/train-test-valid.jpg)

![](https://i.imgur.com/BF9BNR8.gif)



Забираем код сниппета из [Робофлоу](https://universe.roboflow.com/mipt4/archistyles/dataset/1) чтобы загрузить датасет в контейнер Collab.

In [ ]:
# Создаем и переходим в директорию для загрузки датасета
import os
os.makedirs("/content/datasets/", exist_ok=True)
%cd /content/datasets/

/content/datasets


In [ ]:
# Вспомогательный код, чтобы убедиться, что работаешь в правильной папке
# %cd ../datasets/
# print(os.getcwd())

/content/datasets
/content/datasets


In [ ]:
# Устанавливаем робофлоу
!pip install roboflow

from roboflow import Roboflow
from google.colab import userdata

# Путь, где должен находиться локальный датасет
local_dataset_path = "/content/datasets/ArchiStyles-1"

if not os.path.exists(local_dataset_path):
    # Если папки нет, загружаем датасет с Roboflow
    print("Датасет не найден локально. Загружаем с Roboflow...")
    # Вместо userdata.get('roboflow_secret') используйте свой ключ API Roboflow
    # Следующие 4 строки кода из сниппета (Download Dataset) Roboflow
    rf = Roboflow(api_key=userdata.get('roboflow_secret'))
    project = rf.workspace("mipt4").project("archistyles")
    version = project.version(1)
    dataset = version.download("folder")
    dataset_name = dataset.location.split(os.sep)[-1]  # Имя папки с датасетом
else:
    # Если папка есть, просто используем её
    print(f"Датасет найден локально: {local_dataset_path}")
    dataset_name = os.path.basename(local_dataset_path)  # Имя папки с датасетом

# Сохраняем имя датасета в переменную окружения DATASET_NAME
os.environ["DATASET_NAME"] = dataset_name

print(f"Имя датасета сохранено в переменную окружения: {dataset_name}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
Датасет не найден локально. Загружаем с Roboflow...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ArchiStyles-1 in folder:: 100%|██████████| 10653/10653 [00:03<00:00, 2966.39it/s]


Имя датасета сохранено в переменную окружения: ArchiStyles-1


### 2.2 Тренируем модель и сохраняем результаты 🎉
Здесь мы используем переменную окружения `DATASET_NAME` чтобы передать наш датасет в параметр `--data` модели.

> Заметка: здесь мы тренируем модель на 100 эпохах. Также мы используем для старта обучения предтренировочные веса модели YOLOv5. В дальнейшем мы сможем использовать веса результатов уже наших тренировок.


### 2.2.1. Подготовка к сохранению результатов обучения модели

Поскольку Collab стирает все данные при остановке контейнера, мы скопируем результаты обучения на гугл-диск. Для этого:
1. Даем Collab доступ к гугл-диску.
2. Задаем путь к папке сохранения результатов.
3. Создаем название папки для сохранения текущего обучения модели
4. По результату обучения модели копируем результаты обучения из контейнера Collab в создаваемую на этапе 3 папку на гугл-диске.


При завершении обучения YOLOv5 результаты обучения модели будут находятся в папке Collab:

`content/yolov5/runs/train-cls/exp#`,

где `#` - порядковый номер обучения модели). По результату обучения модели мы сохраним их на Google Диске, чтобы коллаб не стер их при остановке контейнера.

### 2.2.2. Подключение к Google Drive и задание пути к папке результатов

In [ ]:
'''
Подключаем гугл-диск для подгрузки данных предыдущих обучений и сохранения
новых результатов.
Ваш гугл-диск будет подключен и синхронизирован с папкой '/content/drive'
этого контейнера Collab
'''
import os
import csv
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive') # путь к папке в контейнере Collab
# При первом запуске нужно будет разрешить Коллабу доступ к Google Drive

'''
  Задайте путь к папке сохранения результатов на вашем Google Drive
  Сохранять нужно сюда: https://drive.google.com/drive/folders/1BfsY5-UEowLiEO6N5yB0mq_B2ZrfeIhh?usp=drive_link
  Часть пути "MyDrive/!!! Data Science/Хакатоны/" у вас будет свой
'''
results_dir = "/content/drive/MyDrive/!!! Data Science/Хакатоны/1_architectural_styles/results"

# проверка корректности пути к папке results_dir
print(os.path.exists(results_dir))

Mounted at /content/drive
True


In [ ]:
# Вспомогательный код
# os.path.exists(results_dir) # проверка папки на существование, корректность пути
# print(os.getcwd()) # проверка в какой мы папке сейчас
# os.listdir() # вывести содержимое текущей папки

True

### 2.2.3. Создание функции генерации имени новой папки сохранения и функций копирования результатов.

In [ ]:
# !pip install pytz # для задания МСК часового пояса
import shutil
import os
import pytz
from datetime import datetime


def save_result_dir_name(results_dir, params):
  '''
  Функция генерирует название для папки на основе параметров обучения.
  @results_dir - путь к папке результатов на гугл-диске
  @params - нужно передать словарь с параметрами обучения модели:
    batch_size, lr, img, epochs, optimizer
  returns: string - возвращает путь к папке сохранения результатов обучения
  '''
  # Создаем временную зону UTC+3
  utc_plus_3 = pytz.timezone("Europe/Moscow")  # UTC+3, соответствует Москве
  # Получаем текущее время с учётом часового пояса
  current_time = datetime.now(utc_plus_3)

  timestamp = current_time.strftime("%Y-%m-%d %H:%M:%S")
  # Создаем название папки на основе параметров обучения модели: время старта + гиперпараметры
  experiment_params = f"{timestamp} batch-size {params['batch_size']} lr {params['lr']} img {params['img']} epochs {params['epochs']} optimizer {params['optimizer']}"
  return os.path.join(results_dir, experiment_params)


def copy_model_run_result(run_type, to_path):
  '''
  Функция копирования папки результата работы модели в папку to_path
  @run_type - тип запуска модели: train или val
  @to_path - путь сохранения результатов
  '''
  from_path = f'/content/yolov5/runs/{run_type}-cls/'
  if os.path.exists(from_path):
    os.chdir(from_path)
    # Копируем папку с последним результатом
    last_run = from_path + os.listdir()[-1]
    shutil.copytree(os.path.join(last_run), os.path.join(to_path))
  else:
    print(f'Папка {from_path} еще не создана. Сперва отработайте модель.')

# Функция копирования файла
def copy_file(from_path, to_path):
    if os.path.isfile(from_path):  # Проверяем, что путь указывает на файл
        # Создаём целевую папку, если она не существует
        os.makedirs(to_path, exist_ok=True)

        # Копируем файл в целевую папку
        destination_path = os.path.join(to_path, os.path.basename(from_path))
        shutil.copy(from_path, destination_path)

        print(f"Файл {from_path} успешно скопирован в {destination_path}")
    else:
        print(f"Файл {from_path} не найден. Проверьте путь и запустите модель.")

### 2.2.4. Обучение модели

In [ ]:
# Копируем лучшие веса в папку Коллаба 'runs/train-cls/best/best.pt'
# Тут нужно указать путь к лучшим весам на гугл-диске
best_weights_file = '/content/drive/MyDrive/!!! Data Science/Хакатоны/1_architectural_styles/results/2024-12-17 13:08:23 batch-size 64 lr 0.001 img 128 epochs 100 optimizer Adam/weights/best.pt'
to_dir = '/content/yolov5/runs/train-cls/best'
copy_file(best_weights_file, to_dir)

# Задаем гиперпараметры обучения модели
# Можем использовать файл наших лучших весов
best_weights = ''
params = {
  "batch_size": 64,
  "lr": 0.001,
  "img": 300,
  "epochs": 50,
  "workers": 8,
  "device": 0,
  # "pretrained_weights": "weights/yolov5s-cls.pt", # начальные веса YOLOv5
  "pretrained_weights": "/content/yolov5/runs/train-cls/best/best.pt", # наши
  "optimizer": "Adam"
}

# Переходим в папку yolov5 и стартуем обучение
%cd /content/yolov5

# Сборка команды как строки
command = (
    f"python classify/train.py "
    f"--model yolov5s-cls.pt "
    f"--data $DATASET_NAME "
    f"--epochs {params['epochs']} "
    f"--img {params['img']} "
    f"--batch-size {params['batch_size']} "
    f"--lr {params['lr']} "
    f"--exist-ok " # Перезаписывает папку exp новыми результатами
    f"--device {params['device']} "
    f"--workers {params['workers']} "
    f"--optimizer {params['optimizer']} "
    f"--pretrained {params['pretrained_weights']}"
)

# Выполнение команды обучения
print("Запуск обучения...")
!{command}

# Создаем название папки для сохранения результатов обучения
save_res_dir = save_result_dir_name(results_dir, params)
# Копируем в нее результаты тренировки
copy_model_run_result(run_type='train', to_path=save_res_dir)
print(f"Результаты скопированы в {save_res_dir}")


Файл /content/drive/MyDrive/!!! Data Science/Хакатоны/1_architectural_styles/results/2024-12-17 13:08:23 batch-size 64 lr 0.001 img 128 epochs 100 optimizer Adam/weights/best.pt успешно скопирован в /content/yolov5/runs/train-cls/best/best.pt
/content/yolov5
Запуск обучения...
2024-12-18 00:37:27.858311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 00:37:27.890379: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 00:37:27.896981: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
classify/train: model=yolov5s-cls.pt, data=ArchiStyles-1, epochs=50, batch_size=64, img

## 3. Валидация модели

In [ ]:
# Копируем лучшие веса в папку Коллаба 'runs/train-cls/best/best.pt'
# Тут нужно указать путь к лучшим весам на гугл-диске
best_weights_file = '/content/drive/MyDrive/!!! Data Science/Хакатоны/1_architectural_styles/results/2024-12-17 13:08:23 batch-size 64 lr 0.001 img 128 epochs 100 optimizer Adam/weights/best.pt'
to_dir = '/content/yolov5/runs/train-cls/best'
copy_file(best_weights_file, to_dir)

# Переходим в папку yolov5 и стартуем валидацию
%cd /content/yolov5

# Задаем параметры валидации
# weights = 'runs/train-cls/best/best.pt' # путь к нашим весам для валидации
weights = 'runs/train-cls/exp/weights/best.pt' # путь к последним весам
dataset = '/content/datasets/$DATASET_NAME' # путь к датасету

# Сборка команды
command = (
    f"python classify/val.py "
    f"--weights {weights} "
    f"--data {dataset}"
)

# Выполнение команды обучения
print("Запуск валидации...")
!{command}
print("Валидация завершена")

# Сохраняем результаты валидации в save_res_dir или custom_dir
custom_dir = '/content/drive/MyDrive/!!! Data Science/Хакатоны/1_architectural_styles/results/2024-12-17 13:08:23 batch-size 64 lr 0.001 img 128 epochs 100 optimizer Adam'
copy_model_run_result(run_type='val', to_path = save_res_dir + '/val')
print(f"Результаты скопированы в {save_res_dir}")

Файл /content/drive/MyDrive/!!! Data Science/Хакатоны/1_architectural_styles/results/2024-12-17 13:08:23 batch-size 64 lr 0.001 img 128 epochs 100 optimizer Adam/weights/best.pt успешно скопирован в /content/yolov5/runs/train-cls/best/best.pt
/content/yolov5
Запуск валидации...
classify/val: data=/content/datasets/ArchiStyles-1, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4180779 parameters, 0 gradients, 10.4 GFLOPs
testing:   0% 0/4 [00:00<?, ?it/s]/content/yolov5/classify/val.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type != "cpu"):
testing: 100% 4/4 [00:03<00:00,  1.16it/s]
           

## 4. Предсказание

In [ ]:
# Копируем тестовую картинку в папку Коллаба 'runs/predict'
img_name = 'roman_building.jpg'
img_path = '/content/drive/MyDrive/!!! Data Science/Хакатоны/1_architectural_styles/images/'
to_dir = '/content/yolov5/runs/predict'
copy_file(img_path+img_name, to_dir)

# Путь к картинке, которую предсказываем
test_img = f'{to_dir}/{img_name}'
# Зададим веса для предсказания
weights = 'runs/train-cls/best/best.pt' # путь к нашим весам для валидации
# weights = 'runs/train-cls/exp/weights/best.pt'

# Сборка команды
command = (
    f"python classify/predict.py "
    f"--weights {weights} "
    f"--source {test_img}"
)

# Переходим в папку yolov5 и стартуем предсказание
%cd /content/yolov5
print("Запуск предсказания...")
!{command}
print("Предсказание готово")

# Сохраняем результаты предсказания
copy_model_run_result(run_type='predict', to_path = save_res_dir + '/predict')
print(f"Результаты скопированы в {save_res_dir}")

Файл /content/drive/MyDrive/!!! Data Science/Хакатоны/1_architectural_styles/images/roman_building.jpg успешно скопирован в /content/yolov5/runs/predict/roman_building.jpg
/content/yolov5
Запуск предсказания...
classify/predict: weights=['runs/train-cls/best/best.pt'], source=/content/yolov5/runs/predict/roman_building.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4180779 parameters, 0 gradients, 10.4 GFLOPs
image 1/1 /content/yolov5/runs/predict/roman_building.jpg: 224x224 Romanesque 0.75, Gothic 0.07, Byzantine 0.04, Beaux-Arts 0.02, Postmodern 0.02, 3.0ms
Speed: 0.3ms pre-process, 3.0ms inference, 26.1ms NMS per image at shape (1, 3, 224, 224)
Results saved to run

We can see the inference results show ~3ms inference and the respective classes predicted probabilities.



---


## (Опционально) Улучшение модели с помощью активного обучения

Now that we've trained our model once, we will want to continue to improve its performance. Improvement is largely dependent on improving our dataset.

We can programmatically upload example failure images back to our custom dataset based on conditions (like seeing an underrpresented class or a low confidence score) using the same `pip` package.

In [ ]:
# # Upload example image
# project.upload(image_path)


In [ ]:
# # Example upload code
# min_conf = float("inf")
# for pred in results:
#     if pred["score"] < min_conf:
#         min_conf = pred["score"]
# if min_conf < 0.4:
#     project.upload(image_path)

###Directory Example

In [ ]:
#Directory infer
os.environ["TEST_CLASS_PATH"] = test_class_path = os.path.join(*os.environ["TEST_IMAGE_PATH"].split(os.sep)[:-1])
print(f"Infering on all images from the directory {os.environ['TEST_CLASS_PATH']}")
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /$TEST_CLASS_PATH/